In [2]:
N = ord('D') % 5 + 1
print(N)

4


In [1]:
import cv2
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd


excel_file = 'lab6.xlsx'
try:
    df = pd.read_excel(excel_file)
except FileNotFoundError:
    raise FileNotFoundError(f"File {excel_file} is not found!")


def color_from_text(color_text):
    color_map = {
        'red': (255, 0, 0),
        'blue': (0, 0, 255),
        'cian': (0, 255, 255),
        'magenta': (255, 0, 255),
        'yellow': (255, 255, 0)
    }
    return color_map.get(color_text.lower(), (0, 0, 0))  


def add_glasses_and_resize(image, output_size, glasses_color, line_width):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        
       
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        if len(eyes) >= 2:
            eyes = sorted(eyes, key=lambda e: e[0])
            pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            draw = ImageDraw.Draw(pil_image)
            eye_centers = []
            radii = []
            for (ex, ey, ew, eh) in eyes[:2]:  
                eye_center = (x + ex + ew // 2, y + ey + eh // 2)
                eye_centers.append(eye_center)
                radius = max(ew, eh) // 2
                radii.append(radius)
                draw.ellipse(
                    [
                        (eye_center[0] - radius, eye_center[1] - radius),
                        (eye_center[0] + radius, eye_center[1] + radius),
                    ],
                    outline=glasses_color,
                    width=line_width,
                )
            
            if len(eye_centers) == 2:
                left_edge = (eye_centers[0][0] + radii[0], eye_centers[0][1])
                right_edge = (eye_centers[1][0] - radii[1], eye_centers[1][1])
                draw.line([left_edge, right_edge], fill=glasses_color, width=line_width)
                

                left_ear_x = eye_centers[0][0] - 2 * radii[0]
                draw.line(
                    [(eye_centers[0][0] - radii[0], eye_centers[0][1]),
                     (left_ear_x, eye_centers[0][1])],
                    fill=glasses_color,
                    width=line_width,
                )
                

                right_ear_x = eye_centers[1][0] + 2 * radii[1]
                draw.line(
                    [(eye_centers[1][0] + radii[1], eye_centers[1][1]),
                     (right_ear_x, eye_centers[1][1])],
                    fill=glasses_color,
                    width=line_width,
                )
            
           
            image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
            break  

    
    height, width = image.shape[:2]
    new_width, new_height = output_size
    
    
    if height > new_height or width > new_width:
        scale_factor = min(new_height / height, new_width / width)
        new_dim = (int(width * scale_factor), int(height * scale_factor))
        image = cv2.resize(image, new_dim, interpolation=cv2.INTER_AREA)
    
    
    height, width = image.shape[:2]
    x_offset = (new_width - width) // 2
    y_offset = (new_height - height) // 2

   
    centered_image = np.ones((new_height, new_width, 3), dtype=np.uint8) * 255

    
    centered_image[y_offset:y_offset+height, x_offset:x_offset+width] = image
    
    return centered_image


for index, row in df.iterrows():
    image_file = row['file name']
    try:
        output_size = tuple(map(int, row['image size'].split('x')))
        glasses_color = color_from_text(row['glasses color'])
        line_width = int(row['line width'])

        
        original_image = cv2.imread(image_file)
        if original_image is None:
            print(f"File {image_file} is not found, skip...")
            continue

      
        processed_image = add_glasses_and_resize(original_image, output_size, glasses_color, line_width)

        
        output_path = f"processed_{image_file}"
        cv2.imwrite(output_path, processed_image)
        print(f"Result saved: {output_path}")
    except Exception as e:
        print(f"Error {image_file}: {e}")


Result saved: processed_emma-watson2.jpg
Result saved: processed_emma-watson.jpg
Result saved: processed_draco.jpg
Result saved: processed_longbottom.jpg
Result saved: processed_ron_wesley.jpg
